# Toxicity detection models
Unigram logistic regression model and CNN-based model. Trained on DKHate.

In [1]:
# Set seed
import random
import numpy as np
import tensorflow as tf # tf backend

# Imports
from danlp.models.embeddings  import load_wv_with_gensim
from gensim.models import KeyedVectors
import json
from keras import layers
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, plot_model
import matplotlib.pyplot as plt
import nltk
import os
import pandas as pd
import pickle
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from statistics import mode
from tqdm import tqdm
import utils

## Load data

In [2]:
# Load data
X_train_orig, X_test_orig, y_train_orig, y_test_orig = utils.load_dkhate(test_size=0.2)

pd.DataFrame([X_train_orig, y_train_orig]).T.tail()

,tweet,label
id,,
2378,Hørt!,0
1879,Min reaktion hvis jeg var Svensker,0
42,hey champ Vil du ikke smide et link til hvor d...,0
457,Hvis du melder en for vold eller voldtægt og d...,1
3108,"Vi betaler for omkring 4mb ned, 500kb op men f...",0


## Preprocessing

In [3]:
stop_words = nltk.corpus.stopwords.words('danish')

# preprocess train and test set
X_train_preproc = X_train_orig.apply(lambda text: utils.preprocess(text, stopwords=stop_words, to_string=True))
X_test_preproc = X_test_orig.apply(lambda text: utils.preprocess(text, stopwords=stop_words, to_string=True))

print("Before preprocessing:")
display(pd.DataFrame([X_train_orig, y_train_orig]).T.tail())
print(f"Vocab size: {len(utils.get_vocab(X_train_orig, are_tokens=False)):,}")

print("\n\nAfter preprocessing:")
display(pd.DataFrame([X_train_preproc, y_train_orig]).T.tail())
print(f"Vocab size: {len(utils.get_vocab(X_train_preproc, are_tokens=False)):,}")

Before preprocessing:


,tweet,label
id,,
2378,Hørt!,0
1879,Min reaktion hvis jeg var Svensker,0
42,hey champ Vil du ikke smide et link til hvor d...,0
457,Hvis du melder en for vold eller voldtægt og d...,1
3108,"Vi betaler for omkring 4mb ned, 500kb op men f...",0


Vocab size: 13,117


After preprocessing:


,tweet,label
id,,
2378,hørt,0
1879,reaktion svensker,0
42,hey champ smide link ser hearthstone henne,0
457,melder vold voldtægt viser sandt beviser diver...,1
3108,betaler omkring mb kb får nok tættere kb kb be...,0


Vocab size: 8,918


## Test other models

In [5]:
from danlp.models import load_bert_offensive_model

# load the offensive language identification model
offensive_model = load_bert_offensive_model()

In [6]:
sentence = "Kriminelle damer"

# apply the model on the sentence to get the class in which it belongs
pred = offensive_model.predict(sentence)
# or to get its probability of being part of each class
proba = offensive_model.predict_proba(sentence)
print(proba[1])

0.16129124


In [7]:
sentence = "Kriminelle herrer"

# apply the model on the sentence to get the class in which it belongs
pred = offensive_model.predict(sentence)
# or to get its probability of being part of each class
proba = offensive_model.predict_proba(sentence)
print(proba[1])

0.1257779


Also test two other models: https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/hatespeech.md